### instantiating services

In [1]:
from apps.homebase.abis import wrapperAbi, daoAbiGlobal,tokenAbiGlobal
from apps.homebase.paper import Paper
from datetime import datetime
import time
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
import re
cred = credentials.Certificate('homebase.json')
initialize_app(cred)
db = firestore.client()
networks=db.collection("contracts")
ceva=networks.document("Etherlink-Testnet").get()
wrapper_address = ceva.to_dict()['wrapper']
print("wrapper address :" + str(wrapper_address))
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
papers={}
daos=[]
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

daos_collection=db.collection('idaosEtherlink-Testnet')
docs = list(daos_collection.stream())
dao_addresses = [doc.id for doc in docs]

for doc in docs:
    obj=doc.to_dict()
    try:
        p=Paper(address=obj['token'],kind="token", daos_collection=daos_collection,db=db,  web3=web3, dao=doc.id)
        dao=Paper(address=obj['address'],kind="dao", daos_collection=daos_collection,db=db,  web3=web3, dao=doc.id)
    except Exception as e:
        print("one DAO contract can't parse correctly: "+str(e))
    papers.update({obj['token']:p})
    papers.update({obj['address']:dao})

event_signatures = {
    web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256,address,string[],string[])").hex(): "NewDaoCreated",
    "0x01c5013cf023a364cc49643b8f57347e398d2f0db0968edeb64e7c41bf2dfbde":"NewDaoCreated",
    "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f":"DelegateChanged",
    "0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0":"ProposalCreated",
    "0x9a2e42fd6722813d69113e7d0079d3d940171428df7373df9c7f7617cfda2892":"ProposalQueued",
    "0x712ae1383f79ac853f8d882153778e0260ef8f03b504e2866e0593e04d2b291f":"ProposalExecuted",
    "0xb8e138887d0aa13bab447e82de9d5c1777041ecd21ca36ba824ff1e6c07ddda4":"VoteCast"
}

papers.update({wrapper_address:Paper(address=wrapper_address,kind="wrapper",daos_collection=daos_collection,db=db,  web3=web3 )})
listening_to_addresses=[wrapper_address]
listening_to_addresses=listening_to_addresses+list(papers.keys())

counter=0
processed_transactions = set()
heartbeat=0
print(f"Listening for {len(event_signatures)} events on {len(papers.items())} contracts...")

while True:
    heartbeat+=1
    try:
        latest=web3.eth.block_number
        first=latest-4
        logs = web3.eth.get_logs({
            "fromBlock": first,
            "toBlock": latest,
            "address": listening_to_addresses,
            # "topics": [[*event_signatures.keys()], None]
        })
        for log in logs:
            tx_hash = log["transactionHash"].hex()
            if tx_hash in processed_transactions:
                print("already did this one")
                continue  # Skip duplicate
            contract_address = log["address"]
            event_signature = log["topics"][0].hex()
            event_name = event_signatures[f"0x{event_signature}"]
            print(f"Event: {event_name}, Contract: {contract_address}")
            new_contract_addresses=papers[contract_address].handle_event(log, func=event_name)
            if new_contract_addresses != None:
                dao_address=new_contract_addresses[0]
                token_address=new_contract_addresses[1]
                listening_to_addresses=listening_to_addresses+[dao_address]+[token_address]
                papers.update({token_address:Paper(address=token_address,kind="token", dao=dao_address, web3=web3 )})
                papers.update({dao_address:Paper(address=dao_address,kind="dao", dao=dao_address, web3=web3 )})
            processed_transactions.add(tx_hash)
    except Exception as e:
        print("something went wrong "+str(e))

    if heartbeat%50==0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)
    

wrapper address :0xDF38eD7d9DcF15577F0E6eb619b945669F1f74b1
node connected
Listening for 7 events on 5 contracts...
heartbeat: 50


KeyboardInterrupt: 

In [3]:
import apps.homebase.entities as entities
import importlib
importlib.reload(entities)

latest_block = 14511276
from_block = latest_block - 2
ceva=""
logs = web3.eth.    get_logs({
    "fromBlock": from_block,
    "toBlock": latest_block,
    "address": "0x09166B4cA9572648B42A2A644D0c0daF4888Df48",
    # "topics": [[*event_signatures.keys()],None]

})
if len(logs)>0:
    event_signature = log["topics"][0].hex()
    ceva=event_signature
for log in logs:
    # Decode the log
    event = web3.eth.contract(abi=daoAbiGlobal).events.ProposalExecuted().process_log(log)  
len(logs)


1

In [6]:
proposal_id = str(event['args']['proposalId'])
proposal_id

'38802198531964484310600104438794817880476366815233449201654127116345723490739'

In [20]:
project_addresses = [doc.id for doc in projects_collection.stream()]
project_addresses

['0x379080EE8941CE5Ae5560d5E32A4CEBBaC7C6b92']

In [10]:
a=[1,2,3,4,5,6,7,8,9]
a[-2]

8

In [ ]:

def listen_to_project_events(contract, contract_address):
    global processed_transactions
    
    # Define the topics for all events
    event_signatures = {
        "SetParties": web3.keccak(text="SetParties(address,address,string)").hex(),
        "SendFunds": web3.keccak(text="SendFunds(address,uint256)").hex(),
        "ContractorPaid": web3.keccak(text="ContractorPaid(address,uint256)").hex(),
        "ContributorWithdrawn": web3.keccak(text="ContributorWithdrawn(address,uint256)").hex(),
        "ProjectDisputed": web3.keccak(text="ProjectDisputed(address)").hex(),
        "ProjectClosed": web3.keccak(text="ProjectClosed(address)").hex(),
        "ContractSigned": web3.keccak(text="ContractSigned(address)").hex(),
        "ArbitrationDecision": web3.keccak(text="ArbitrationDecision(address,uint256,string)").hex(),
    }

    # Get logs for the current contract
    logs = web3.eth.get_logs({
        "fromBlock": web3.eth.block_number-3,
        "toBlock": web3.eth.block_number,
        "address": contract_address,
    })

    for log in logs:
        tx_hash = log["transactionHash"].hex()
        if tx_hash in processed_transactions:
            print("already did this one")
            continue  # Skip already processed transactions

        try:
            event_type = None
            for event_name, topic_hash in event_signatures.items():
                print("got one!")
                if log["topics"][0].hex() == topic_hash:
                    event_type = event_name
                    break

            if not event_type:
                print("Not an avent type")
                continue  # Skip logs that don't match any defined events

            # Decode and process the event
            decoded_event = getattr(contract.events, event_type)().process_log(log)
            event_data = getattr(ProjectEvents, event_type)(decoded_event)

            print(f"Event detected in contract {contract_address} ({event_type}):", event_data)

        except Exception as e:
            print(f"Error processing log for contract {contract_address}: {e}")

        # Mark the transaction as processed
        processed_transactions.add(tx_hash)


In [ ]:
while True:
    heartbeat += 1

    for address in project_addresses:
        # Get contract instance
        contract = web3.eth.contract(address=address, abi=native_project_abi)
        try:
            listen_to_project_events(contract, address)
        except Exception as e:
            
            print("Errrorrr "+e)

    if heartbeat % 10 == 0:
        print(f"Heartbeat: {heartbeat}")

    time.sleep(3)

In [ ]:
import apps.services

In [ ]:
import requests
dorg_homebase_channel_id = "804689936388325376"

def send_discord_message(msg, channel_id):
    global discord_bot_token
    url = "https://discordapp.com/api/channels/"+channel_id+"/messages"
    headers = {
        "Authorization": "Bot " + discord_bot_token
    }
    body = {
        "content": msg
    }
    print("before sending "+msg)
    response = requests.request("POST", url, headers=headers, data=body)
    return {"data": str("Status code "+str(response.status_code))}

In [31]:
send_discord_message(channel_id=dorg_homebase_channel_id, msg="INDEXER IS DOWN ON TRUSTLESS!")

before sending INDEXER IS DOWN ON TRUSTLESS!


{'data': 'Status code 200'}

In [33]:
from dotenv import dotenv_values
env_variables = dotenv_values(".env")

In [ ]:
my_variable = env_variables["discord_bot_token"]
print(my_variable)